# GEOG 489 Project

## Introduction

### 1. Motivations

Food safety is a critical public health issue. In urban areas like Chicago, Illinois, government oversight of institutions that serve food to the public is important to maintain the health of the people who eat or acquire food at these institutions. Failure to follow proper health and safety codes on the part of private enterprises can lead to outbreaks of infectious and food-borne illnesses.

### 2. Goals

The City of Chicago plots every health inspection of a food-serving institution on a map along with the institution’s evaluated risk level (Low, Medium, High). By analyzing this data, we will be able to visualize which areas might be underserved and therefore at higher risk for poor food safety and increased risk of food-borne illnesses. We are interested to see if there is a correlation between risk level and demographic data, and will create visualizations to see how this changes. The available dataset has a decade of health inspection data, so we will be able to see if there are any long term trends, in regards to what kind of establishments are inspected more frequently, and what association there is with the demographics of the region. 


Additionally, we will plot these inspection records on the Chicago road network, which will allow us to perform route efficiency analysis to maximize the number of inspections a theoretical inspector could perform. We will perform a network analysis using the restaurants as nodes, to create a visualization of the nearest restaurants to a health inspection office. By optimizing the route a theoretical health inspector might take, we could improve the efficiency of health inspections and potentially service more restaurants on a daily basis. 


### 3. Datasets

- Shapefile of Chicago
- Food inspections: This dataset is derived from inspections of restaurants and other food establishments in Chicago from January 1, 2010 to the present. Inspections are performed by staff from the Chicago Department of Public Health’s Food Protection Program using a standardized procedure.
- Census data: Selected socioeconomic indicators in Chicago
- Chicago road networks

![jupyter](./figures/road_networks.png)

## Implementation

### 0. Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import time
import plotly.express as px
import seaborn
from dash import Dash, dcc, html, Input, Output
from collections import defaultdict
import warnings
import libpysal
import esda
warnings.filterwarnings('ignore')

### 1. Import Data

In [ ]:
# Import shapefile
chicomm = gpd.read_file('./data/chicomm/chicomm.shp')
chicomm.head()

In [ ]:
# Import food inspection data
inspections = pd.read_csv('./data/Food_Inspections_2018to22.csv')
inspections.head()

In [ ]:
# Import census data
census = pd.read_csv('./data/Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012.csv')
census.head()

In [ ]:
# Overview of the data
# Chicago shapefile
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
chicomm.plot(ax=ax)

In [ ]:
# Pie chart: risk level
inspections_by_risk = inspections.groupby('Risk').size()
plt.pie(inspections_by_risk,
        labels=inspections_by_risk.index.to_list(),
        radius=2);

In [ ]:
# Pie chart: inspection type
inspections_by_type = inspections.groupby('Inspection Type').size()
plt.pie(inspections_by_type,
        labels=inspections_by_type.index.to_list(),
        radius=2);

### 2. Data Preparation

In [ ]:
# Prepare for merging
chicomm = chicomm[['DISTNAME', 'DISTITLE', 'geometry']]
chicomm.dtypes

In [ ]:
# Census data for each community
census_comm = census[:77]
census_chi = census[77:]

# Drop a useless column
census_comm = census_comm.drop(columns=['COMMUNITY AREA NAME'])

# Check data type
census_comm.dtypes

In [ ]:
# Convert data type for merging
census_comm['Community Area Number'] = census_comm['Community Area Number'].astype(int)

In [ ]:
chicomm = chicomm.merge(census_comm, left_on='DISTNAME', right_on='Community Area Number')
chicomm.head()

In [ ]:
# Reorganize inspection data
inspections = inspections.drop(columns=['Location'])

inspections['Inspection Date'] = inspections['Inspection Date'].map(lambda x: time.strptime(x, "%m/%d/%Y"))

In [ ]:
inspections = gpd.GeoDataFrame(
    inspections, geometry=gpd.points_from_xy(inspections.Longitude, inspections.Latitude))

inspections = inspections.drop(columns=['Latitude', 'Longitude'])
inspections['Inspection Date']

The module "time" in Python uses structures like above to record date and time. Attributes: tm_year, tm_mon, tm_mday, tm_hour, tm_min. tm_sec, tm_wday, tm_yday and tm_isdst. We only care about the date.

Now we need to organize the inspection data by inspection date. This step can be quite time consuming if we keep the geodataframe data structure. So we use list instead.

In [ ]:
inspections = inspections.set_crs('epsg:4269')
inspections_by_year = defaultdict(list)

In [ ]:
# For each row in the inspection dataset, we convert it to a list
for idx, row in inspections.iterrows():
    inspections_by_year[row['Inspection Date'][0]].append(row.tolist())

Convert 2-D lists to geodataframes.

In [ ]:
for key in inspections_by_year:
    inspections_by_year[key] = gpd.GeoDataFrame(inspections_by_year[key], columns=inspections.columns)
    inspections_by_year[key] = inspections_by_year[key].set_crs('epsg:4269')
    
inspections_by_year[2020].head()

### 3. Dynamic Choropleth Maps

There are several ways to make choropleth maps in python. In our project, we will look into choropleth maps in two modules: geopandas and plotly.

#### 3.1 Choropleth maps in geopandas

In [ ]:
# Spatial join
total_inspections = gpd.sjoin(inspections, chicomm[['DISTITLE', 'geometry']], op='within')
high_risk = gpd.sjoin(inspections[inspections['Risk']=='Risk 1 (High)'], chicomm[['DISTITLE', 'geometry']], op='within')

total_inspections_comm = total_inspections.groupby('DISTITLE').size()
high_risk_comm = high_risk.groupby('DISTITLE').size()

total_inspections_comm = total_inspections_comm.to_frame(name='Number of Total Inspections')
high_risk_comm = high_risk_comm.to_frame(name='Number of High Risk')

chicomm_sjoin = chicomm.merge(total_inspections_comm, left_on='DISTITLE', right_on='DISTITLE')
chicomm_sjoin = chicomm_sjoin.merge(high_risk_comm, left_on='DISTITLE', right_on='DISTITLE')

In [ ]:
# High risk rate
chicomm_sjoin['Rate of High Risk'] = chicomm_sjoin.apply(lambda x: x['Number of High Risk']/x['Number of Total Inspections'], axis=1)

In [ ]:
chicomm_sjoin.head()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 10))

ax[0].set_title('Total Inspections')
ax[1].set_title('Number of High Risk')
ax[2].set_title('Rate of High Risk')

chicomm.boundary.plot(ax=ax[0], color='black')
chicomm.boundary.plot(ax=ax[1], color='black')
chicomm.boundary.plot(ax=ax[2], color='black')

chicomm_sjoin.plot(column='Number of Total Inspections',
                   ax=ax[0],
                   cmap='Blues',
                   legend=True,
                   scheme='FisherJenks',
                   k=5)

chicomm_sjoin.plot(column='Number of High Risk',
                   ax=ax[1],
                   cmap='Reds',
                   legend=True,
                   scheme='FisherJenks',
                   k=5)

chicomm_sjoin.plot(column='Rate of High Risk',
                   ax=ax[2],
                   cmap='Reds',
                   legend=True,
                   scheme='FisherJenks',
                   k=5);

#### 3.2 Choropleth maps in plotly

First, let's take a look at an example of choropleth maps in plotly.express.

Overview: The plotly.express module (usually imported as px) contains functions that can create entire figures at once, and is referred to as Plotly Express or PX. Plotly Express is a built-in part of the plotly library, and is the recommended starting point for creating most common figures. More details: https://plotly.com/python/plotly-express/

We can use API plotly.express.choropleth to make choropleth maps:

In [ ]:
# chicomm_sjoin = chicomm_sjoin.set_index('DISTITLE')

fig = px.choropleth(chicomm_sjoin,
                    geojson=chicomm_sjoin.geometry,
                    locations=chicomm_sjoin.index,
                    color='Rate of High Risk',
                    color_continuous_scale="Reds",
                    projection="mercator")

fig.update_geos(fitbounds="locations", visible=False)

fig.update_layout(
    title_text='Rate of High Risk'
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_layout(
    coloraxis_colorbar={'title':''})

fig.show()

In [ ]:
inspections_by_year.keys()

In [ ]:
# Organize data of 13 (4) years 
merged_data = gpd.GeoDataFrame()
for key in sorted(inspections_by_year):
    curr_data = inspections_by_year[key]
    total = gpd.sjoin(curr_data, chicomm[['DISTITLE', 'geometry']], op='within')
    high_risk = gpd.sjoin(curr_data[inspections['Risk']=='Risk 1 (High)'], chicomm[['DISTITLE', 'geometry']], op='within')

    total_comm = total.groupby('DISTITLE').size()
    high_risk_comm = high_risk.groupby('DISTITLE').size()

    total_comm = total_comm.to_frame(name='Number of Total Inspections')
    high_risk_comm = high_risk_comm.to_frame(name='Number of High Risk')

    curr_data_sjoin = chicomm.merge(total_comm, left_on='DISTITLE', right_on='DISTITLE')
    curr_data_sjoin = curr_data_sjoin.merge(high_risk_comm, left_on='DISTITLE', right_on='DISTITLE')

    curr_data_sjoin['Rate of High Risk'] = curr_data_sjoin.apply(lambda x: x['Number of High Risk']/x['Number of Total Inspections'], axis=1)
    curr_data_sjoin['Year'] = key
    merged_data = merged_data.append(curr_data_sjoin)

In [ ]:
merged_data.head()

In [ ]:
# Dynamic map with timeline
# merged_data = merged_data.set_index('DISTITLE')
fig = px.choropleth(merged_data,
                    geojson=merged_data.geometry,
                    locations=merged_data.index,
                    color='Rate of High Risk',
                    color_continuous_scale="Reds",
                    animation_frame='Year',
                    projection="mercator")

fig.update_geos(fitbounds="locations", visible=False)

fig.update_layout(
    title_text='Rate of High Risk'
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_layout(
    coloraxis_colorbar={'title':''})

fig.show()

### 4. Correlation Analysis

In [ ]:
seaborn.pairplot(chicomm_sjoin,
                 x_vars=['PERCENT OF HOUSING CROWDED',
                         'PERCENT HOUSEHOLDS BELOW POVERTY',
                         'PER CAPITA INCOME ',
                         'PERCENT AGED 16+ UNEMPLOYED',
                         'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA'],
                 y_vars='Rate of High Risk',
                 kind='reg',
                 height=5
                );

### 5. Spatial Autocorrelation

In [ ]:
w_queen = libpysal.weights.Queen.from_dataframe(chicomm_sjoin[['Rate of High Risk', 'geometry']])
w_rook = libpysal.weights.Rook.from_dataframe(chicomm_sjoin[['Rate of High Risk', 'geometry']])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20,10))

chicomm_sjoin.boundary.plot(ax=axes[0], ls=':', color='black')
w_rook.plot(chicomm_sjoin, ax=axes[0], 
            edge_kws=dict(color='r', linestyle=':', linewidth=1),
            node_kws=dict(marker=''))

chicomm_sjoin.boundary.plot(ax=axes[1], ls=':', color='black')
w_queen.plot(chicomm_sjoin, ax=axes[1], 
             edge_kws=dict(color='r', linestyle=':', linewidth=1),
             node_kws=dict(marker=''))

In [ ]:
y = chicomm_sjoin['Rate of High Risk']

mi_rook = esda.moran.Moran(y, w_rook)
mi_queen = esda.moran.Moran(y, w_queen)
print(f"Moran's I with Rook's case contiguity: {round(mi_rook.I, 3)}, p-value: {round(mi_rook.p_norm, 3)}")
print(f"Moran's I with Queen's case contiguity: {round(mi_queen.I, 3)}, p-value: {round(mi_queen.p_norm, 3)}")